# Historical Introduction (Apostol, Calculus Vol. 1)

## Scope
This notebook follows the main themes under "Historical Introduction" and turns them into small computational/visual experiments and structured notes.

## Topics
1. The two basic concepts of calculus
2. Historical background
3. The method of exhaustion for the area of a parabolic segment
4. Exercises (referenced by number only)
5. A critical analysis of Archimedes’ method
6. The approach to calculus used in this book


# 1. The Two Basic Concepts of Calculus
# Two geometric problems that motivate calculus

A useful way to motivate calculus is to start from two closely related geometric questions.  
First, given a curve $y=f(x)$ drawn above a horizontal baseline and an interval $[a,b]$, we want to associate a single number with the region bounded by the curve, the baseline, and the vertical lines $x=a$ and $x=b$. This motivates the notion of area and leads naturally to the idea of an integral. Second, given a point $x=c$ on the same curve, we want to associate a single number with the “steepness” of the tangent line at that point. This motivates the notion of instantaneous rate of change and leads to the idea of a derivative. In the formal development, these two measurements—area and steepness—become the organizing problems around which integral and differential calculus are built (Adapted from the motivating discussion in Apostol, *Calculus*, Vol. 1, 2nd ed., p. 2.).

In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ipywidgets import FloatSlider, VBox, HBox, Output, Layout
from IPython.display import display, clear_output

# -----------------------------
# Global styling (robust, academic, readable)
# -----------------------------
plt.rcParams.update({
    "font.family": "STIXGeneral",   # ships with Matplotlib, consistent
    "mathtext.fontset": "stix",
    "font.size": 22,
})

# Output area for live updates
out = Output()

# -----------------------------
# Curve (clean sine; always above baseline)
# -----------------------------
def curve(x, A=0.9, w=1.0, phi=0.0, shift=1.6):
    return shift + A * np.sin(w * x + phi)

def numeric_slope(f, c, h=1e-4):
    return (f(c + h) - f(c - h)) / (2 * h)

def clamp(val, lo, hi):
    return max(lo, min(hi, val))

def render(
    A=0.9, w=1.0, phi=0.0, shift=1.6,
    a=1.0, b=6.0, c=7.2,
    xmin=0.0, xmax=10.0,
    fig_path="assets/figures/figure1.png"
):
    # Ensure [a,b] is ordered and inside [xmin,xmax]
    a_, b_ = (a, b) if a <= b else (b, a)
    a_ = clamp(a_, xmin, xmax)
    b_ = clamp(b_, xmin, xmax)

    # Keep c inside [xmin,xmax]
    c_ = clamp(c, xmin, xmax)

    # Functions + data
    x = np.linspace(xmin, xmax, 2500)
    f = lambda t: curve(t, A=A, w=w, phi=phi, shift=shift)
    y = f(x)

    # Measurements: area and slope
    x_area = np.linspace(a_, b_, 15000)
    area = np.trapezoid(f(x_area), x_area)

    m = numeric_slope(f, c_)
    yc = f(c_)
    tangent = yc + m * (x - c_)

    with out:
        clear_output(wait=True)

        # IMPORTANT: Jupyter scales overly wide figures down.
        # Use a balanced size so it stays large on-screen.
        fig, ax = plt.subplots(figsize=(12.5, 8.0), dpi=150)

        # Clean axes
        for s in ["top", "right", "bottom", "left"]:
            ax.spines[s].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])

        # Axis arrows
        ax.annotate("", xy=(xmax, 0), xytext=(xmin, 0),
                    arrowprops=dict(arrowstyle="->", lw=1.9))
        ax.annotate("", xy=(0, max(y) * 1.12), xytext=(0, 0),
                    arrowprops=dict(arrowstyle="->", lw=1.9))

        ax.text(xmax, -0.20, r"$x$", ha="right", va="top", fontsize=26)
        ax.text(-0.20, max(y) * 1.12, r"$y$", ha="right", va="bottom", fontsize=26)

        # Curve C
        ax.plot(x, y, lw=3.2)
        xC = xmin + 0.48 * (xmax - xmin)
        ax.text(xC, f(xC) + 0.20, r"$C$", fontsize=28)

        # Shaded area on [a,b]
        mask = (x >= a_) & (x <= b_)
        ax.fill_between(x[mask], 0, y[mask], alpha=0.18)
        ax.vlines([a_, b_], [0, 0], [f(a_), f(b_)], lw=2.2)

        # Tangent line
        ax.plot(x, tangent, lw=2.6)

        # Tangent label placed to avoid overlap
        x_lab = clamp(c_ + 0.9, xmin + 0.5, xmax - 1.2)
        y_lab = yc + m * (x_lab - c_)
        ax.text(x_lab, y_lab + 0.18, "Line tangent to $C$", fontsize=18, ha="left")

        # Tangency point + direction arrow
        ax.scatter([c_], [yc], s=90, zorder=3)
        dx = 0.9
        ax.annotate("", xy=(c_ + dx, yc + m * dx), xytext=(c_, yc),
                    arrowprops=dict(arrowstyle="->", lw=1.7))
        ax.text(c_ + dx + 0.18, yc + m * dx + 0.04, "steepness", fontsize=18)

        # Labels for a, b, c
        ax.text(a_, -0.34, r"$a$", ha="center", va="top", fontsize=24)
        ax.text(b_, -0.34, r"$b$", ha="center", va="top", fontsize=24)
        ax.text(c_, -0.34, r"$c$", ha="center", va="top", fontsize=24)

        # Limits
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(-0.60, max(y) * 1.18)

        # Compact info box (readable, not intrusive)
        box = (
            r"$A(a,b)\approx \int_a^b f(x)\,dx \approx$ " + f"{area:.4f}\n"
            r"$m(c)\approx f'(c)\approx$ " + f"{m:.4f}\n\n"
            "Apostol, Calculus Vol. 1 (2nd ed.), p. 2"
        )
        ax.text(0.06, 0.92, box, transform=ax.transAxes,
                ha="left", va="top", fontsize=17,
                bbox=dict(boxstyle="round,pad=0.55", fc="0.98", ec="0.35", lw=1.0))

        # Caption (academic, short)
        caption = (
            "Figure 1. Two basic measurements: "
            r"$A(a,b)=\int_a^b f(x)\,dx$ and $m(c)=f'(c)$. "
            "Apostol (Vol. 1, 2nd ed.), p. 2."
        )
        fig.subplots_adjust(bottom=0.20)
        fig.text(0.07, 0.04, caption, fontsize=18)

        # Save figure (always)
        os.makedirs(os.path.dirname(fig_path), exist_ok=True)
        fig.savefig(fig_path, dpi=300, bbox_inches="tight")

        plt.show()

        # Table (rounded, clean)
        df = pd.DataFrame({
            "Parameter": [
                "A (amplitude)", "w (frequency)", "phi (phase)", "shift",
                "a", "b", "c",
                "Area  A(a,b)", "Slope m(c)"
            ],
            "Value": [
                A, w, phi, shift,
                a_, b_, c_,
                area, m
            ]
        })
        df["Value"] = df["Value"].astype(float).round(6)
        display(df)

# -----------------------------
# Controls (sliders)
# -----------------------------
slider_layout = Layout(width="260px")

sA     = FloatSlider(value=0.9, min=0.2, max=1.6, step=0.05, description="A",     continuous_update=False, layout=slider_layout)
sw     = FloatSlider(value=1.0, min=0.4, max=2.0, step=0.05, description="w",     continuous_update=False, layout=slider_layout)
sphi   = FloatSlider(value=0.0, min=-3.14, max=3.14, step=0.05, description="phi", continuous_update=False, layout=slider_layout)
sshift = FloatSlider(value=1.6, min=0.6, max=3.0, step=0.05, description="shift", continuous_update=False, layout=slider_layout)

sa = FloatSlider(value=1.0, min=0.0, max=9.0, step=0.1, description="a", continuous_update=False, layout=slider_layout)
sb = FloatSlider(value=6.0, min=0.0, max=9.0, step=0.1, description="b", continuous_update=False, layout=slider_layout)
sc = FloatSlider(value=7.2, min=0.0, max=10.0, step=0.1, description="c", continuous_update=False, layout=slider_layout)

controls1 = HBox([sA, sw, sphi, sshift])
controls2 = HBox([sa, sb, sc])

def _update(*args):
    render(
        A=sA.value, w=sw.value, phi=sphi.value, shift=sshift.value,
        a=sa.value, b=sb.value, c=sc.value,
        fig_path="assets/figures/figure1.png"
    )

for wdg in [sA, sw, sphi, sshift, sa, sb, sc]:
    wdg.observe(_update, names="value")

display(VBox([controls1, controls2, out]))

# initial render
_update()


Calculus, at its core, is concerned with the precise formulation and solution of two fundamental geometric measurement problems: the assignment of a number to represent the area of a region and the assignment of a number to represent the steepness of a tangent line at a point on a curve. In modern language, these two measurements are expressed by the integral and the derivative, which allow us to define the concepts of area and tangent line and to compute, respectively, the area of a given region and the slope of a given tangent. The historical origin of integral calculus goes back more than two thousand years to the Greek method of exhaustion, in which one approximates a given region by inscribed polygonal regions whose areas can be computed exactly, then improves the approximation by using polygons with more and more sides in an attempt to “exhaust” the region. This idea was used with remarkable success by Archimedes to obtain exact formulas for the area of the circle and other special figures, but further development was impossible at the time due to the absence of a suitable algebraic language. The elementary algebra familiar today did not exist in antiquity, and without efficient symbolic notation it was practically impossible to extend the method to general classes of regions. A slow but revolutionary transformation began in the sixteenth century with the replacement of Roman numerals by Hindu–Arabic numerals, the introduction of the symbols + and −, and the growing acceptance of decimal notation; during the same period, the work of Tartaglia, Cardano, and Ferrari on cubic and quartic equations greatly advanced algebraic methods and encouraged the adoption of a more powerful symbolic language. With the widespread use of well-chosen algebraic symbols, interest in the ancient method of exhaustion was revived, leading to many partial results by Cavalieri, Torricelli, Roberval, Fermat, Pascal, and Wallis, and gradually the method evolved into what is now called integral calculus, a new and powerful discipline with applications far beyond geometry. Its greatest impetus came in the seventeenth century through the work of Isaac Newton and Gottfried Wilhelm Leibniz, and its rigorous foundations were established in the nineteenth century by mathematicians such as Augustin-Louis Cauchy and Bernhard Riemann, while further refinements and extensions of the theory continue in contemporary mathematics.